#### Modelo de recomendacion ítem-ítem, donde se toma un item, en base a que tan similar es ese ítem al resto, para recomendar ítems similares.

In [1]:
# se importan librerias

import pandas as pd
import numpy as np
import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import operator
import fastparquet as fp 
import pyarrow as pa
import pyarrow.parquet as pq

#### Se crea una matriz de variables dummy para representar la presencia o ausencia de ciertas características para cada elemento, para póder calcular la similitud de coseno entre vectores, y estos vectores representaran elementos en un espacio con caracteristicas similares.

In [22]:
# se carga el dataframe en la variable modelado

modelado= pd.read_parquet("Data/steam_games.parquet")

modelado.head()

,genres,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,Año_estreno,publisher,app_name,title,developer
0,Action,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,4.49,['Single-player'],4.99,False,761140,NaN,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
1,Casual,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,4.49,['Single-player'],4.99,False,761140,NaN,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
2,Indie,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,4.49,['Single-player'],4.99,False,761140,NaN,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
3,Simulation,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,4.49,['Single-player'],4.99,False,761140,NaN,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
4,Strategy,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,4.49,['Single-player'],4.99,False,761140,NaN,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro


In [3]:
modelado.columns

Index(['genres', 'url', 'release_date', 'tags', 'reviews_url',
       'discount_price', 'specs', 'price', 'early_access', 'id', 'metascore',
       'Año_estreno', 'publisher', 'app_name', 'title', 'developer'],
      dtype='object')

In [23]:
# Seleccionar las columnas
columnas_seleccionadas = modelado[['id','title','genres']]

# Copiar las columnas seleccionadas en otra variable
modelado_item = columnas_seleccionadas.copy()

modelado_item

,id,title,genres
0,761140,Lost Summoner Kitty,Action
1,761140,Lost Summoner Kitty,Casual
2,761140,Lost Summoner Kitty,Indie
3,761140,Lost Summoner Kitty,Simulation
4,761140,Lost Summoner Kitty,Strategy
...,...,...,...
71546,610660,Russian Roads,Indie
71547,610660,Russian Roads,Racing
71548,610660,Russian Roads,Simulation
71549,658870,EXIT 2 - Directions,Casual


In [24]:
duplicates = modelado_item[modelado_item.duplicated()]

# Mostrar las filas duplicadas (si las hay)
if not duplicates.empty:
    print("Filas duplicadas encontradas:")
    print(duplicates)
else:
    print("No hay filas duplicadas en el DataFrame.")

No hay filas duplicadas en el DataFrame.


In [25]:
# Selecciona las primeras 25055 filas del DataFrame por la limitacion de la capacidad de computo del equipo donde se esta trabajando, esa es la cantidad con la que puede correr al aplicar 
# la funcion "similitud del coseno"

modelado_item = modelado_item.head(25055)

modelado_item

,id,title,genres
0,761140,Lost Summoner Kitty,Action
1,761140,Lost Summoner Kitty,Casual
2,761140,Lost Summoner Kitty,Indie
3,761140,Lost Summoner Kitty,Simulation
4,761140,Lost Summoner Kitty,Strategy
...,...,...,...
25050,697860,Cathodemer,Animation &amp; Modeling
25051,697860,Cathodemer,Video Production
25052,654175,Blacksquad - S15 FIRST RELEASE PACKAGE,Action
25053,654175,Blacksquad - S15 FIRST RELEASE PACKAGE,Free to Play


In [28]:
# Crear una instancia de CountVectorizer con un límite de 5000 características y usando palabras comunes en inglés como stop words

conteo_vector = CountVectorizer(max_features = 5000, stop_words="english")

In [29]:
# Transformar los datos de la columna "genres" en una matriz de recuento de términos

conteo_vector.fit_transform(modelado_item["genres"]).toarray().shape

(25055, 32)

In [30]:
# Ajustar el modelo CountVectorizer a los datos de la columna "genres" del DataFrame modelado_item
# y transformar los datos en una matriz de recuento de términos.

vectores = conteo_vector.fit_transform(modelado_item["genres"]).toarray()


In [31]:
# se muestra la matriz

vectores

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]], dtype=int64)

In [32]:
# Calcular la similitud coseno entre las filas de la matriz de vectores

similitud = cosine_similarity(vectores)


In [33]:
# se mira la primera fila

similitud[0]

array([1., 0., 0., ..., 1., 0., 1.])

In [34]:
# Ordenar la lista de tuplas (índice, valor de similitud) en orden descendente según el valor de similitud,
# y seleccionar los elementos desde el segundo hasta el sexto en la lista ordenada, excluyendo el primero.

sorted(list(enumerate(similitud[0])), reverse=True, key=lambda x: x[1])[1:6]


[(14, 1.0), (17, 1.0), (36, 1.0), (40, 1.0), (68, 1.0)]

In [35]:
# Definir una función de recomendación que toma el título de un juego como entrada
def recomendacion(game):
    # Obtener el índice del juego en el DataFrame modelado_item
    indice_juegos = modelado_item[modelado_item["title"] == game].index[0]

    # Obtener las distancias de similitud coseno entre el juego dado y todos los demás juegos
    distances = similitud[indice_juegos]

    # Crear una lista de tuplas (índice, valor de similitud) ordenada en orden descendente por similitud,
    # y seleccionar los cinco juegos más similares, excluyendo el propio juego.
    lista_juegos = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]

    # Obtener los títulos de los juegos recomendados a partir de los índices obtenidos en la lista anterior
    recommended_titles = [modelado_item.iloc[i[0]]['title'] for i in lista_juegos]

    # Devolver la lista de títulos recomendados
    return recommended_titles


In [36]:
# muestra las recomendaciones segun titulo del juego

recomendacion('Lost Summoner Kitty')

['弹炸人2222',
 'Battle Royale Trainer',
 'Uncanny Islands',
 'Army of Tentacles: (Not) A Cthulhu Dating Sim: Black GOAT of the Woods Edition',
 'The Warrior Of Treasures']

In [37]:
# Crear una nueva columna 'recomendaciones' en el DataFrame modelado_item
# que contiene las recomendaciones de juegos para cada título, calculadas usando la función recomendacion.

modelado_item['recomendaciones'] = modelado_item['title'].apply(recomendacion)


In [38]:
# muestra el dataframe resultante

modelado_item

,id,title,genres,recomendaciones
0,761140,Lost Summoner Kitty,Action,"[弹炸人2222, Battle Royale Trainer, Uncanny Islan..."
1,761140,Lost Summoner Kitty,Casual,"[弹炸人2222, Battle Royale Trainer, Uncanny Islan..."
2,761140,Lost Summoner Kitty,Indie,"[弹炸人2222, Battle Royale Trainer, Uncanny Islan..."
3,761140,Lost Summoner Kitty,Simulation,"[弹炸人2222, Battle Royale Trainer, Uncanny Islan..."
4,761140,Lost Summoner Kitty,Strategy,"[弹炸人2222, Battle Royale Trainer, Uncanny Islan..."
...,...,...,...,...
25050,697860,Cathodemer,Animation &amp; Modeling,"[3D-Coat V4.8, Axis Game Factory's AGFPRO v3, ..."
25051,697860,Cathodemer,Video Production,"[3D-Coat V4.8, Axis Game Factory's AGFPRO v3, ..."
25052,654175,Blacksquad - S15 FIRST RELEASE PACKAGE,Action,"[弹炸人2222, Battle Royale Trainer, Uncanny Islan..."
25053,654175,Blacksquad - S15 FIRST RELEASE PACKAGE,Free to Play,"[弹炸人2222, Battle Royale Trainer, Uncanny Islan..."


In [39]:
# elimina la columna "title" y "genres" ya no son necesarias

modelado_item = modelado_item.drop(['title','genres'], axis=1)



In [50]:
# Filtrar filas duplicadas en el DataFrame modelado_item basándose en la columna 'id'
# Mantener todas las instancias duplicadas en el conjunto de resultados.

duplicates = modelado_item[modelado_item.duplicated(subset=['id'], keep=False)]

# Mostrar el DataFrame resultante que contiene solo las filas duplicadas.
duplicates



,id,recomendaciones
0,761140,"[弹炸人2222, Battle Royale Trainer, Uncanny Islan..."
1,761140,"[弹炸人2222, Battle Royale Trainer, Uncanny Islan..."
2,761140,"[弹炸人2222, Battle Royale Trainer, Uncanny Islan..."
3,761140,"[弹炸人2222, Battle Royale Trainer, Uncanny Islan..."
4,761140,"[弹炸人2222, Battle Royale Trainer, Uncanny Islan..."
...,...,...
25049,654174,"[弹炸人2222, Battle Royale Trainer, Uncanny Islan..."
25050,697860,"[3D-Coat V4.8, Axis Game Factory's AGFPRO v3, ..."
25051,697860,"[3D-Coat V4.8, Axis Game Factory's AGFPRO v3, ..."
25052,654175,"[弹炸人2222, Battle Royale Trainer, Uncanny Islan..."


In [51]:
# Crear un nuevo DataFrame llamado modelado_item_cleaned, eliminando las filas duplicadas basándose en la columna 'id'
# Mantener solo la primera instancia de cada conjunto de duplicados.

modelado_item_cleaned = modelado_item.drop_duplicates(subset=['id'], keep='first')


In [52]:
# se muestra el dataframe sin duplicados

modelado_item_cleaned

,id,recomendaciones
0,761140,"[弹炸人2222, Battle Royale Trainer, Uncanny Islan..."
5,643980,"[Real Pool 3D - Poolians, SNOW - All Access Ba..."
9,670290,"[Real Pool 3D - Poolians, 弹炸人2222, Race, Army ..."
14,767400,"[弹炸人2222, Battle Royale Trainer, Uncanny Islan..."
17,772540,"[弹炸人2222, Battle Royale Trainer, Uncanny Islan..."
...,...,...
25045,702413,"[Real Pool 3D - Poolians, 弹炸人2222, Race, Army ..."
25048,654174,"[弹炸人2222, Battle Royale Trainer, Uncanny Islan..."
25050,697860,"[3D-Coat V4.8, Axis Game Factory's AGFPRO v3, ..."
25052,654175,"[弹炸人2222, Battle Royale Trainer, Uncanny Islan..."


In [56]:
# Definir una función llamada recomendacion_juego que toma el id de un juego como entrada
def recomendacion_juego(id_producto):
    # Filtrar el DataFrame modelado_item para obtener las recomendaciones de juegos
    # asociadas con el juego específico identificado por su id.
    recomendaciones = modelado_item[modelado_item['id'] == id_producto]['recomendaciones'].iloc[0]

    # Verificar si hay recomendaciones disponibles para el juego dado
    if len(recomendaciones) > 0:
        # Crear un diccionario numerado de recomendaciones para una presentación más organizada
        recomendaciones_dict = {i + 1: juego for i, juego in enumerate(recomendaciones)}
        # Devolver el diccionario de recomendaciones numeradas
        return recomendaciones_dict
    else:
        # Si no hay recomendaciones disponibles, devolver un diccionario vacío
        return {}


In [57]:
# se muestra las recomendaciones segun el id del juego

recomendaciones_dict = recomendacion_juego(643980)
print(recomendaciones_dict)

{1: 'Real Pool 3D - Poolians', 2: 'SNOW - All Access Basic Pass', 3: 'SNOW - All Access Pro Pass', 4: 'SNOW - All Access Legend Pass', 5: 'SNOW - Starter Pack'}


In [55]:
# Especifica la ruta donde deseas guardar el archivo Parquet
ruta_guardado = 'Data/Recomendacion.parquet'

# Guarda el DataFrame en formato Parquet
modelado_item_cleaned.to_parquet(ruta_guardado, index=False)

print(f'DataFrame guardado en: {ruta_guardado}')

DataFrame guardado en: Data/Recomendacion.parquet
